<a href="https://colab.research.google.com/github/NiklasSlager/equdist/blob/master/notebooks/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet -U git+https://github.com/NiklasSlager/equdist.git

In [2]:
import sys
sys.path.append("..")

In [ ]:
from equdist.physical_data_sets import save_component_list

compounds = ['n-propane', 'iso-butane']
save_component_list(compounds)

ModuleNotFoundError: No module named 'equdist'

In [ ]:
import jax.numpy as jnp
from time import time
from jax import jit, vmap
import matplotlib.pyplot as plt
from equdist.plot_generation import plot_function
from equdist.model import inside_simulation as simulation
from equdist.model import initialize
from equdist.model import condensor_duty
import os
from tqdm import tqdm


state_init = initialize(len(compounds))



zf = jnp.array([0.86, 0.14])
zf = zf/jnp.sum(zf)
iterations = 0
st = time()


state = jit(simulation)(
        state=state_init,
        nstages=jnp.array(5, dtype=int),
        feedstage=jnp.array(4, dtype=int),
        pressure=jnp.array(1.0, dtype=float),
        feed=jnp.array(1000.0, dtype=float),
        z=jnp.array(zf, dtype=float),
        distillate=jnp.array(300, dtype=float),
        rr=jnp.array(2.5, dtype=float),
        specs=False
    )

print(state.X[:, 0])
print(state.distillate)

print('evaluation time = ', time()-st)
print('Equimolar NR iterations to convergence =', state.EQU_iterations)
print('Full NR iterations to convergence =', state.NR_iterations)
plot_function(jnp.arange(1, state.Nstages+1), state.L[0:state.Nstages], state.V[0:state.Nstages], state.temperature[0:state.Nstages], state.Hliq[0:state.Nstages], state.Hvap[0:state.Nstages], state.X[jnp.where(zf != 0, jnp.arange(0, len(zf)), 0), 0:state.Nstages], state.Y[jnp.where(zf != 0, jnp.arange(0, len(zf)), 0), 0:state.Nstages], jnp.where(zf != 0, jnp.arange(0, len(zf)), 0), figsize = (12, 5))
#plot_function(jnp.arange(1, state.Nstages[0]+1), state.L[0, 0:state.Nstages[0]], state.V[0, 0:state.Nstages[0]], state.temperature[0, 0:state.Nstages[0]], state.Hliq[0, 0:state.Nstages[0]], state.Hvap[0, 0:state.Nstages[0]], state.X[0, jnp.where(zf != 0, jnp.arange(0, len(zf)), 0), 0:state.Nstages[0]], state.Y[0, jnp.where(zf != 0, jnp.arange(0, len(zf)), 0), 0:state.Nstages[0]], jnp.where(zf != 0, jnp.arange(0, len(zf)), 0))

plt.show()